# Importing Necessary Libraries

In [1]:
# These are used to handle data laoding directly from Kaggle
import requests
from zipfile import ZipFile
from io import BytesIO

# Preprocessing
import re
import numpy as np
import pandas as pd
from dython.nominal import associations
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Modelling, Predication, and Evaluation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

#Creating Machine learning Pipeline for deployment
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Feature Selection & Feature Engineering
# from sklearn.

#Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


KeyboardInterrupt



# Importing Training Data

In [ ]:
# URL of the zip file
zip_url = 'https://storage.googleapis.com/kagglesdsdata/competitions/3316/31070/Train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1716873096&Signature=d%2F%2FVSWINnU0EyUFzDZ1nxlH0lbLWpdBLW0Y7uofGk386wren22IuV4nN0grD9DHPyA08TtXpQuSVCK7%2B6vj%2BW1ixMMTtn67z1f68ib4Z%2BZqniBlLPmTdrFetlGyYlof12gPBL%2B7CAfbMvg7s9Yv%2BCt5UWYEqKj4yz2UcJ7OO0AM80TlKs2B12dd0leFCB4Ud2DcpGwZW%2Fu%2Bdl6Hyr%2F0ws7aUQJP0%2BSC38DcgBLRlxPbhsreUZh10%2BemYLegwfGF8388eW%2BLLQh1Yk9J03%2FCuFkade5jaK1oZXLKnlt2PXkQ5uJ4AiTvGOAKEp5KyhwPvbPUnkgOSc0RqN8Eeg3EHtA%3D%3D&response-content-disposition=attachment%3B+filename%3DTrain.zip'
# Download the zip file
response = requests.get(zip_url)
# Check if the request was successful
if response.status_code == 200:
    # Read the zip file content
    with ZipFile(BytesIO(response.content)) as z:
        # Extract the file names from the zip
        file_list = z.namelist()

        # Assuming there's only one CSV file in the zip archive
        csv_filename = file_list[0]

        # Read the CSV file into a DataFrame
        with z.open(csv_filename) as csv_file:
            df = pd.read_csv(csv_file, low_memory=False, parse_dates=['saledate'])
else:
    print("Failed to download the zip file")

In [ ]:
# Set the display format for floating-point numbers
pd.options.display.float_format = '{:,.2f}'.format
df.head()

# Exploratory Data Analysis (EDA)

In [ ]:
df.info()

## Numeric Attributes

In [ ]:
df.describe().T

In [ ]:
df.select_dtypes(['int', 'float']).isna().sum().sort_values(ascending=False)

In [ ]:
cols_to_del = set()
# Determining features that adds no information to the model
for col in df.columns:
  # Columns that have a different value per sample (Unique Identifiers)
  if len(df[col]) == df[col].nunique():
    cols_to_del.add(col)
  # Columns that have the same value for all samples (Static Value)
  elif df[col].nunique == 1:
    cols_to_del.add(col)
print(cols_to_del)

### Inspecting SalePrice (Target Column)

In [ ]:
# Create a figure and axis object
fig, ax = plt.subplots(figsize=(12, 6))

# Calculate statistics
median_SalePrice = df['SalePrice'].median()
mean_SalePrice = df['SalePrice'].mean()
lower_bound = df['SalePrice'].quantile(0.025)
upper_bound = df['SalePrice'].quantile(0.975)

# Plot histogram
ax = sns.histplot(data=df, x='SalePrice')
ax.axvline(x=median_SalePrice, color='red', linestyle='-', label='Median')
ax.axvline(x=mean_SalePrice, color='green', linestyle='-', label='Mean')
ax.axvspan(lower_bound, upper_bound, alpha= 0.2, color='grey')

# Set labels and title
plt.xlabel('Sale Price')
plt.ylabel('Frequency')
plt.title('Distribution of SalePrice')

# 95% confidence interval area 
ax.text(lower_bound, ax.get_ylim()[1]*0.9, f'Lower Bound\n({lower_bound:,.0f})', ha='right', va='top', rotation=90)
ax.text(median_SalePrice, ax.get_ylim()[1]*0.9, f'Median\n({median_SalePrice:,.0f})', ha='center', va='top', rotation=90)
ax.text(upper_bound, ax.get_ylim()[1]*0.9, f'Upper Bound\n({upper_bound:,.0f})', ha='left', va='top', rotation=90)

plt.legend()

plt.show();

### Inspecting MachineID

In [ ]:
df.sort_values('MachineID').head(2).T

In [ ]:
possible_correlated_cols = ['MachineID', 'ModelID', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'ProductGroup', 'ProductGroupDesc', 'Enclosure']
cat_cols = df[possible_correlated_cols].select_dtypes('object').columns.tolist()

# Compute associations between selected columns
assoc = associations(df[possible_correlated_cols], nominal_columns=cat_cols, plot=False)

# Extract the correlation matrix
corr_matrix = assoc['corr']

# Mask the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Set up the matplotlib figure
plt.figure(figsize=(12, 8))

# Plotting
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={"shrink": .8}, vmin=-1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
cols_to_del.add('MachineID')

### Inspecting datasource

In [ ]:
df.datasource.value_counts(normalize=True).sort_values(ascending=False)

In [ ]:
# Compacting the datasources into 3 category
df.datasource = df.datasource.apply(lambda x: x if (x == 132) | (x == 136) else 'Other')

In [ ]:
# Change data type to category
df.datasource = df.datasource.astype('category')

### Inspecting auctioneerID

In [ ]:
# since auctioneerID has missing values, it should be further investigated prior to making any changes
df.auctioneerID.unique()

In [ ]:
print(f"Mode of Auctionner ID: {df.auctioneerID.mode()[0]}")
print(f"Median of Auctioneer ID: {df.auctioneerID.median()}")

In [ ]:
df.auctioneerID.value_counts(normalize=True, dropna=False).sort_values(ascending=False)

In [ ]:
sns.scatterplot(data=df, x='auctioneerID', y='SalePrice', alpha=0.1);

In [ ]:
# Step1: Check to see whether auctioneerID no. 99 is a form of missing value

#first we set aside the actual missing values 
df_temp = df.dropna(subset='auctioneerID')

mask = df_temp.auctioneerID == 99

# Extracting data for each group
id99 = df_temp[mask]['SalePrice']
others = df_temp[~mask]['SalePrice']

# Create a new DataFrame for boxplot and histplot
boxplot_data = pd.DataFrame({
    'SalePrice': pd.concat([id99, others]),
    'Group': ['ID no. 99'] * len(id99) + ['Other IDs'] * len(others)
})

# Plotting
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

# Histpgram
sns.histplot(others, ax=ax[0], alpha=0.3, stat='density', color='r', label='Other IDs', kde= True)
sns.histplot(id99, ax=ax[0], alpha=0.3, stat='density', color='b', label='ID no. 99', kde= True)
ax[0].legend()
ax[0].set_title('SalePrice Distribution')

# Boxplot
sns.boxplot(x='SalePrice', y='Group', data=boxplot_data, ax=ax[1], orient='h', showmeans=True)
ax[1].set_title('Boxplot of SalePrice')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

Although the mean of the two groups seem to be different, the histogram and boxplot of the two groups show a great overlap so it seems that there is no significant differnece between the mean of Sale Price in these two groups. However, to confirm our conclusion, we run the `Wilcoxon` test on these two groups. The reason we choose `Wilcoxon` over the two-sample `t-test` is that two assumptions of the `t-test` is not met here:
1. Normality
2. Homoscedasticity

Our Hypotheses are as follows:

- H0: $μ_{1}$ = $μ_{2}$
- H1: $μ_{1}$ ≠ $μ_{2}$

In [ ]:
# Confirming our conclusion using statistical analysis (Wilcoxon test)
u_stat, p_val_mw = mannwhitneyu(others, id99, alternative='two-sided')
if p_val_mw < 0.05:
    print('H0 is rejected. AuctioneerID 99 should be treated as a missing value.')
else:
    print('There is no significant difference and auctioneerID 99 should be treated as a valid id.')

Well that's contradictory to what we expected. We calculate the effect size (e.g., Cohen’s d) to understand the magnitude of the difference between the two groups. Even a small p-value might correspond to a negligible effect size, especially in large datasets.
Cohen's d is a measure of effect size used to indicate the standardized difference between two means. The interpretation of Cohen's d is somewhat subjective, but commonly used thresholds are:

- 0.2: Small effect
- 0.5: Medium effect
- 0.8: Large effect

In [ ]:
# Calculate Cohen's d
def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / dof)
    return (np.mean(x) - np.mean(y)) / pooled_std

effect_size = cohen_d(others, id99)
effect_size

Given the effect size of 0.184:
1. **Small Effect:** This is close to the threshold of 0.2, which means the difference between the group with auctioneerID 99 and other IDs is small.
2. **Practical Implication:** Even though the wilcoxon indicates a statistically significant difference (p-value < 0.05), the small effect size suggests that the practical difference between the two groups is minor.

Thus, we decide to treat auctioneer id 99 as a valid id. However, one can run a sensetivity analysis on either case to see which results in a better prediction performance.

In [ ]:
# Step2: Check to see which type of missing value are we dealing with in auctioneerID
mask = df.auctioneerID.isna()

# Extracting data
mask = df.auctioneerID.isna()
MV = df[mask]['SalePrice']
WMV = df[~mask]['SalePrice']

# Create a new DataFrame for boxplot and histplot
boxplot_data = pd.DataFrame({'SalePrice': pd.concat([MV, WMV]),
                            'Labels':['With MV']*len(MV)+['Without MV']*len(WMV)})

# Plotting
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

# Histogram
sns.histplot(MV, label='With MV', stat='density', color='red', kde=True, alpha= 0.3, ax=ax[0])
sns.histplot(WMV, label='Without MV', stat='density', color='blue', kde=True, alpha= 0.3, ax=ax[0])
ax[0].legend()
ax[0].set_title('SalePrice Distribution')

# Boxplots
sns.boxplot(x= 'SalePrice', y='Labels', data=boxplot_data, ax=ax[1], orient='h', showmeans=True)
ax[1].set_title('Boxplot of SalePrice')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

Same as the previous one, the histogram and the boxplot of the two groups show a great overlap so it seems that there is no significant differnece between the mean of Sale Price in these two groups. However, to confirm our conclusion, we run the `Wilcoxon` test on these two groups. The reason we choose `Wilcoxon` over the two-sample `t-test` is that two assumptions of the `t-test` is not met here:
1. Normality
2. Homoscedasticity

Our Hypotheses are as follows:

- H0: $μ_{1}$ = $μ_{2}$
- H1: $μ_{1}$ ≠ $μ_{2}$

In [ ]:
u_stat, p_val_mw = mannwhitneyu(WMV, MV, alternative='two-sided')

if p_val_mw < 0.05:
    print('H0 is rejected. Missing values are of type MNAR and thus should be discarded.')
else:
    print('There is no significant difference and the missing values are of type MCAR and should be imputed.')

We are experiencing the same situation as above, so once again we calculate the effect size of this difference:

In [ ]:
cohen_d(WMV, MV)

Given the effect size of 0.067:
1. **Small Effect:** The difference between the group with missing values and the rest is small.
2. **Practical Implication:** Even though the wilcoxon indicates a statistically significant difference (p-value < 0.05), the small effect size suggests that the practical difference between the two groups is minor.

Thus, since the nature of the missing values are of type MCAR, we decide to keep them as is; that's because considering the value_counts of IDs and the nominal nature of this feature, apart from IDs ranging from 1-3, we can mark the rest as `Other`s without imputing the missing values and introducing bias.

In [ ]:
# Replacing values other than 1-3 with Other
df.auctioneerID = df.auctioneerID.apply(lambda x: x if x in range(1, 4) else 'Other')

In [ ]:
# Changing datatype to Category
df.auctioneerID = df.auctioneerID.astype('category')

### Inspecting YearMade

In [ ]:
df.YearMade.describe()

In [ ]:
# Turning YearMade = 1000 into NAN values
df.loc[:, 'YearMade']= df['YearMade'].apply(lambda x: np.nan if x == 1000 else x)

In [ ]:
df[df.YearMade>2011]

In [ ]:
# training data are supposed to be up to the end of the 2011, so a YearMade beyound that is not possible
df['YearMade'] = df['YearMade'].apply(lambda x: np.nan if x > 2011 else x)

In [ ]:
df.YearMade.isna().sum()

In [ ]:
# Checking to see if the missing values in the YearMade are MCAR, MAR, or MNAR

# Extracting data
mask = df.YearMade.isna()
MV = df[mask]['SalePrice']
WMV = df[~mask]['SalePrice']

# Create a new DataFrame for boxplot and histplot
boxplot_data = pd.DataFrame({'SalePrice': pd.concat([MV, WMV]),
                             'Labels': ['Missing']*len(MV)+['Available']*len(WMV)})

# Plotting
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

# Histogram
sns.histplot(MV, label='Year Missing', kde=True, stat='density', color='red', ax=ax[0], alpha= 0.3)
sns.histplot(WMV, label='Year Available', kde=True, stat='density', color='blue', ax=ax[0], alpha=0.3)
ax[0].set_title('SalePrice Distribution per YearMade Status')
ax[0].legend()

# Boxplot
sns.boxplot(x='SalePrice', y='Labels', data=boxplot_data, orient='h', showmeans=True)
ax[1].set_title('SalePrice Boxplot per YearMade Status')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show();

While analysing the dataset, we suspected whether Machines with the same `ModelID` number are built around the same Year. If so, we can leverage this finding to impute our missing `YearMade` values with the median of those years. Thus, We need to perform two actions now:
1. Confirm our hypothesis throughout the whole dataset
2. Perform Imputation

In [ ]:
# Checking if our hypothesis is correct by calculating the min, max, and range of YearMade for each ModelID
model_year_stats = df.groupby(by='ModelID')['YearMade'].agg(['max', 'min', 'median', 'mean', pd.Series.mode]).reset_index()
model_year_stats['range'] = model_year_stats['max'] - model_year_stats['min']
model_year_stats['mean-median'] = model_year_stats['median'] - model_year_stats['mean']

# Set a threshold for the range (e.g., 5 years)
threshold = 5

model_year_stats[model_year_stats['mean-median'] > threshold].sort_values('mean-median')

To impute the missing values in the `YearMade` column based on the median `YearMade` for each `ModelID`, we take the following steps:
1. Group by ModelID to calculate the median YearMade for each group.
2. Create a mapping from ModelID to its corresponding median YearMade.
3. Apply this mapping to fill in the missing values in the YearMade column.
4. Compare the Histogram of `YearMade` before and after the imputation to make sure the distribution of data has not changed significantly.

In [ ]:
# Step 1: Group by ModelID and calculate the median YearMade for each group
model_year_medians = df.groupby(['ModelID'])['YearMade'].median()

# Step 2: Create a mapping from ModelID to its corresponding median YearMade
model_year_mapping = model_year_medians.to_dict()

# Step 3: Apply the mapping to fill in the missing values in the YearMade column
temp_year = df.apply(lambda row: model_year_mapping[row['ModelID']] if pd.isna(row['YearMade']) else row['YearMade'], axis=1)
temp_year = temp_year.fillna(np.median(temp_year.dropna()))

# Step 4: Plot both distributions together for comparison
fig, ax = plt.subplots(figsize=(12, 6))
sns.histplot(x='YearMade', data=df, kde=True, bins= 30, color='green', alpha=0.3, label='Before Imputation')
sns.histplot(temp_year, kde=True,  bins= 30, color='blue', alpha=0.3, label='After Imputation')
plt.xlabel('YearMade')
plt.ylabel('Frequency')
plt.legend()
plt.title('Distribution of YearMade Before and After Imputation')
plt.show()

In [ ]:
# Replacing the imputed values
df['YearMade'] = temp_year

### Inspecting MachineHoursCurrentMeter

In [ ]:
# First turning all 0s to null according to the data dictionary of the dataset
df['MachineHoursCurrentMeter'] = df['MachineHoursCurrentMeter'].replace(0, np.nan)

In [ ]:
df['MachineHoursCurrentMeter'].describe()

In [ ]:
# Checking the Outliers
df_temp = df[["MachineHoursCurrentMeter", "YearMade"]].copy()

df_temp["ReportDate"] = 2012
df_temp["24h"] = 365 * 24 * (df_temp["ReportDate"] - df_temp["YearMade"])
df_temp["Suspicious_MHCM"] = df_temp["MachineHoursCurrentMeter"]>df_temp["24h"]

# Identifying records that state the machine has worked more than 24h per day during their lifespan
MHCM_outliers = df_temp[df_temp["Suspicious_MHCM"] == True].sort_values(['YearMade']).index.tolist()

# Replacing with NAN values
df.loc[MHCM_outliers, 'MachineHoursCurrentMeter'] = np.nan

In [ ]:
df.MachineHoursCurrentMeter.describe()

To impute the missing values in the `MachineHoursCurrentMeter` column based on the median value of `MachineHoursCurrentMeter` grouped by `ModelID` and `YearMade`, we take the following steps:

1. Group the data by ModelID and YearMade to calculate the median MachineHoursCurrentMeter for each group.
2. Create a mapping from the combination of ModelID and YearMade to their corresponding median MachineHoursCurrentMeter.
3. Apply this mapping to fill in the missing values in the MachineHoursCurrentMeter column.
4. Compare the Histogram of MachineHoursCurrentMeter before and after the imputation to make sure the distribution of data has not changed significantly.

In [ ]:
# Step 1: Group by ModelID and YearMade and calculate the median MachineHoursCurrentMeter for each group
overall_median_hours = df['MachineHoursCurrentMeter'].median()
grouped = df.groupby(['ModelID', 'YearMade'])['MachineHoursCurrentMeter'].median().reset_index()
grouped = grouped.dropna(subset=['MachineHoursCurrentMeter'])

# Step 2: Create a dictionary mapping from (ModelID, YearMade) to median MachineHoursCurrentMeter
median_mapping = grouped.set_index(['ModelID', 'YearMade'])['MachineHoursCurrentMeter'].to_dict()

# Step 3: Define a function to fill missing values using the mapping
def impute_machine_hours(row):
    if pd.isna(row['MachineHoursCurrentMeter']):
        return median_mapping.get((row['ModelID'], row['YearMade']), overall_median_hours)
    else:
        return row['MachineHoursCurrentMeter']
# Apply the function to the DataFrame
machine_hour_temp = df.apply(impute_machine_hours, axis=1)

# Step 4: Plot both distributions together for comparison
fig, ax = plt.subplots(figsize=(12, 6))
sns.histplot(x='MachineHoursCurrentMeter', data=df, stat='density', kde=True, bins= 30, color='green', alpha=0.3, label='Before Imputation')
sns.histplot(machine_hour_temp, stat='density', kde=True,  bins= 30, color='blue', alpha=0.3, label='After Imputation')
plt.xlabel('MachineHoursCurrentMeter')
plt.ylabel('Density')
plt.legend()
plt.title('Distribution of MachineHoursCurrentMeter Before and After Imputation')
plt.show()

In [ ]:
df.MachineHoursCurrentMeter = machine_hour_temp

### Inspecting saledate

In [ ]:
# Generating Year & Month from saledate
df['SaleYear'] = df.saledate.dt.year
df['SaleMonth'] = df.saledate.dt.month

In [ ]:
df.head()

In [ ]:
cols_to_del.add('saledate')

## Categorical Columns

In [ ]:
cat_cols = df.select_dtypes('object').copy()
cat_cols.describe().T

In [ ]:
cat_cols.isna().sum().sort_values(ascending=False)

### Inspecting 'fiModelDesc', 'fiBaseModel' , 'fiSecondaryDesc' , 'fiModelSeries' , 'fiModelDescriptor'

In [ ]:
cat_cols[['fiModelDesc', 'fiBaseModel' , 'fiSecondaryDesc' , 'fiModelSeries' , 'fiModelDescriptor']].iloc[3:7,:]

In [ ]:
# Check the variety of the rest of the columns according to the ModelID
df.groupby(['ModelID'])[['fiBaseModel' , 'fiSecondaryDesc' , 'fiModelSeries' , 'fiModelDescriptor']].nunique()

In [ ]:
cols_to_del.update(['fiModelDesc', 'fiBaseModel' , 'fiSecondaryDesc' , 'fiModelSeries' , 'fiModelDescriptor'])

### Inspecting UsageBand

In [ ]:
df_temp = df[['ModelID', 'YearMade', 'MachineHoursCurrentMeter' , 'UsageBand']].sort_values(['ModelID'])
df_temp.head()

In [ ]:
sns.scatterplot(x=df.MachineHoursCurrentMeter, y=df.YearMade, hue=df.UsageBand);

Considering the logic behind how the UsageBand is determined, regardless of high number of missing values we decide to impute them through classification

In [ ]:
# Separating data into train & test
mask = df_temp.UsageBand.isna()
train_set = df_temp[~mask]
test_set = df_temp[mask]

X_train = train_set.drop('UsageBand', axis=1)
y_train = train_set['UsageBand']
X_test = test_set.drop('UsageBand', axis=1)
y_test = test_set['UsageBand']

# Standardized data
scaler= StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train & evaluate the model on training data
knn = KNeighborsClassifier(n_neighbors=3)
cross_val_score(knn, X_train_scaled, y_train, cv=3)

In [ ]:
# Predicting UsageBand
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
y_pred

In [ ]:
# Replacing missing values with predicted values
df.loc[df.UsageBand.isna(), 'UsageBand'] = y_pred

In [ ]:
ordinal_mapping = {'Low': 3, 'Medium': 2, 'High': 1}

# Encode the labels using the mapping
df['UsageBand'] = df['UsageBand'].map(ordinal_mapping)

### Inspecting ProductGroup, ProductGroupDesc, fiProductClassDesc, & ProductSize

In [ ]:
df.ProductGroupDesc.unique()

In [ ]:
df.ProductGroup.unique()

#### Product Group
There are 6 types of Bulldozers in this dataset:
1. **Wheel Loader:** This machine has large wheels and is used for scooping and transporting materials. It features a front-mounted bucket.
2. **Skid Steer Loaders:** These are small, versatile loaders with lift arms that can attach to various tools. They are known for their maneuverability and compact size.
3. **Track Excavators:** Also known simply as excavators, these machines have tracks for movement and a large arm with a bucket for digging and moving earth.
4. **Backhoe Loaders:** These machines combine a loader on the front and a backhoe on the back, making them ersatile for various digging and loading tasks.
5. **Track Type Tractors:** Commonly referred to as bulldozers, these are powerful machines with large tracks and a wide blade for pushing earth.
6. **Motor Graders:** These machines have a long blade used for grading surfaces, often used in road construction and maintenance.

There are no missing values in either `ProductGroupDesc` or `ProductGroup` and since ProductGroup is the abbreviate version of ProductGroupDesc, we keep `ProductGroup` and discard ProductGroupDesc.

In [ ]:
cols_to_del.add('ProductGroupDesc')

#### fiProductClassDesc

In [ ]:
df.fiProductClassDesc.head()

General Size Categories Based on Horsepower and Operating Capacity

**Horsepower (HP) Ranges:**
- **Compact:** 0 to 40 HP
- **Mini:** 40 to 60 HP
- **Small:** 60 to 100 HP
- **Medium:** 100 to 200 HP
- **Large:** 200 to 500 HP
- **Very Large:** 500+ HP

**Operating Capacity (Weight) Ranges:**
- **Compact:** 0 to 701 lbs
- **Mini:** 701 to 1251 lbs
- **Small:** 1251 to 1751 lbs
- **Medium:** 1751 to 2701 lbs
- **Large:** 2701+ lbs

**Metric Tons:**
- **Compact:** 0 to 2 metric tons
- **Mini:** 2 to 4 metric tons
- **Small:** 4 to 8 metric tons
- **Medium:** 8 to 20 metric tons
- **Large:** 20 to 50 metric tons
- **Very Large:** 50+ metric tons

**Steps:**
1. Extracting the configuration data

In [ ]:
# Function to extract numbers and specific text patterns from text
def extract_numbers(text):
    # Use regular expression to find all numeric values
    numbers = re.findall(r'\d+\.?\d*', text)
    # Convert the extracted strings to int
    numbers = [float(num) for num in numbers]
        
    # Define specific text patterns to search for
    specific_texts = ['Horsepower', 'Metric Tons', 'Operating Capacity', 'Digging Depth']
    # Use regular expression to find specific text patterns
    found_texts = [pattern for pattern in specific_texts if re.search(pattern, text, re.IGNORECASE)]
    return numbers + found_texts

df['config'] = df['fiProductClassDesc'].apply(extract_numbers)

In [ ]:
df.config = df.config.apply(lambda x: np.nan if len(x) == 0 else x)

In [ ]:
cols_to_del.add('fiProductClassDesc')

#### ProductSize
- **Wheel Loaders**:
  - **Compact**: these are small, highly maneuverable loaders used in tight spaces and smaller projects*
    - **Horsepower**: Up to 40 
    - ** **Operating Capacy*** **Up to 2000 s**
    - **Metric Tons: **Up to 1on**
  - **Smal HP; used for light to medium-duty sks.*
    - **Horsepower**: 40-100 HP
    - **Operating Capacy**: **2000-50 lbs**
    -  *Metric ns**: **2 tons**
  - **M-200 HP; versatile machines used in a variety of appcations.*
    - **Horsepower**: 100-200 HP
    - **Operating Cacity**: **50010000 lbs**
    - **Metc Tons****2-5 tons**
  *200-350 HP; used for heavy-duty tasks in constructi and mining.*
    - **Horsepower**: 200-350 HP
    - **Operati Capacity**: **000-20000 lbs**
    - *etric Ton*: **5-10 tons**
  -*: *Over 350 HP; used in large-scale mining and hvy construction.*
    - **Horsepower**: Over 350 HP
    - **Opeting Capacy**: **20000+ lbs**
    **Metrio- **: **10+ tons**

- **Skid Steer Loadermpact**: *Under 40 HP; designed for light-duty tasks and maneuverabity in confined spaces.*
    - **Horsepower**: Up to 40 HP
    -*Operating Capity**: **Up to 1500 lbs*
    - **Metricons**: **Up to 0  - **Small**: *40-100 HP; more powerfulstill highly maneuverable.*
    - **Horsepower**: 40-100 HP
  - **OperatinCapacity**: **1500-3000 s**
    - **tric Tons**: **0.
  - **Medium**: *100-200 HP; larger models for more demanding tasks, though skid steers are generally on the smaller end of the size spectm compared to other equipment.*
    - **Horsepower**: 100-200 H
    - **Opering Capacity**: **3000-50 lbs**
   
- **ric Tons**: **1.:**.5 tons**

- tors**:
  - **Mini*r 40 HP; also referred to as mini or compact excavators, used for sall-scale tasks and in tight spaces.*
    - **Horsepower**: Up t40 HP
    - *perating Capacity**: **Uto 2000 lbs
    - **Metricp to 1 ton**
  - **Small: *40-100 HP; for small to medium tasks.*
    - **Horsepower**40-100 HP
  - **Operating Capacity****2000-50 lbs**
    - **: **1-2 tons**
  - **Medium**: *100-200 HP; genel-purpose excavators for a variety of tasks.*
    - **Horsepowe*: 100-200 HP   - **Operating Capacit*: **50010000 lbs**
   ons**: **2-5 tons**
  - **Large**: *20350 HP; for heavy construction and mining tasks.*
    - **Horsepower**: 200-350 HP
    - **Operating Capaty**: **100-20000 lbs**
    -**: **5-10 tons**
  - **Very Large**:Over 350 HP; for large-scale mining and earthmoving.*
    - **Hoepower**: er 350 HP
    - **Operang Capac**: **20000+ lbs**
    - **Metric Tons**

- **Backhoe Loaders**: - **Compact**: *Under 40 HP; smaller, versatile machines.*
    **Horsepower**Up to 40 HP
    - **Opeting Capaci**: **Up to 2000- **Metric Tons**: **Up to 1 ton**
  - **Small**: *40-10HP; used for a variety of tasks on smaller construction sites.*    - **Horsewer**: 40-100 HP
    - OperatinCapacity**: **200
    - **Metric Tons**: **1-2 tons**
- **Medium**: *100-200 HP; larger models for more demanding task*
    - **Horpower**: 100-200 HP
    **Operag Capacity**: **5000-10000 lbs**
    - **Metric Tons**:

- **Track Type Tractors (Bulldozers)**:
  - **Smll**: *100-200 HP; used for residential and light construction pjects.*
    - Horsepower**: 100-200 HP    - **Orating Capacity**00 lbs**
    - **Metric Tons**: **5-10 tons**
  - *Medium**: *200-350 HP; versatile machines for medium to heavy cstruction.*
  - **Horsepower**: 200-35HP
    - Operating Capaci-40000 lbs**
    - **Metric Tons**: **10-20 ts**
  - **Large**: *350-500 HP; used in large construction projts and mining.*    - **Horsepower**: 35500 HP
  - **Operating Capacit0000 lbs**
    - **Metric Tons**: **20-30 to**
  - **Very Large**: *Over 500 HP; for the largest constructioand miningasks.*
    - **Horsepow**: Over0 HP
    - **Operating Capacity**: **
    - **Metric Tons**: **30+ tons**

- **Motor aders**:
  - **Medium**: *100-200 HP; commonly used in road conruction and maienance.*
    - **Horseper**: 10000 HP
    - **Ocity**: **10000-20000 lbs**
    - **Metric Tons**: **5-10 tons**
  - **Larg*: *200-350 HP; used for larger road construction projects and  er heavy-duty gding tasks.*
    - **Hoepower**: 0-350 HP
    - **Opey**: **20000-40000 lbs**
    - **Metric Tons**: **10-2 tons**
  - **Very Large**: *Over 350 HP; used in large-scale cotruction a mining operations.*
  - **Horso0-350 HP`; used for larger road construction projects and other heavy-duty grading tasks.
    - **Very Large:** `Over 350 HP`; used in large-scale construction and mining operations.



Steps we are going to take her
1. Decide whether we can merge `Small`, `Compact`, and `Mini` as one
2. Turning theas `Large/Mediu category to either `Large` or `Medium` according to `HorsePower` or `Operating Capacity` and d remove `Large/Medium` catego
3. . Decide how to deal with missing values

In [ ]:
df.ProductSize.value_counts(dropna=False)

In [ ]:
# Understanding the distribution of Product Size over Product Group
pd.crosstab(df.ProductSize, df.ProductGroupDesc, dropna=False, margins=False)

In [ ]:
# Checking on the distribution of `Small`, `Mini`, & `Compact` sizes over Product Group
temp = df[df.ProductSize.isin(['Small', 'Mini', 'Compact'])]
pd.crosstab(temp.ProductSize, temp.ProductGroupDesc)

In [ ]:
# Check to see if we can merge `Small`, `Mini`, & `Compact` sizes
mini = df[(df.ProductSize == 'Mini')&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']
small = df[(df.ProductSize == 'Small')&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']
compact = df[(df.ProductSize == 'Compact')&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']

box_data = pd.DataFrame({'Price Range': pd.concat([mini, small, compact]),
                         'Size': ['Mini']*len(mini) + ['Small']*len(small) + ['Compact']*len(compact)})

plt.figure(figsize=(12, 6))
sns.boxplot(x='Price Range', y='Size', data=box_data)
plt.ylabel('')
plt.title('Price Comparison for Track Excavators in `Mini` & `Small` Sizes');

In [ ]:
# Checking on Missing Values in `Track Excavators` Group
mini = df[(df.ProductSize == 'Mini')&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']
small = df[(df.ProductSize == 'Small')&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']
large = df[(df.ProductSize == 'Large / Medium')&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']
missing = df[(df.ProductSize.isna())&(df.ProductGroupDesc == 'Track Excavators')]['SalePrice']

box_data = pd.DataFrame({'Price Range': pd.concat([mini, small, large, missing]),
                         'Size': ['Mini']*len(mini) + ['Small']*len(small) + ['Large']*len(large) + ['Missing']*len(missing)})

plt.figure(figsize=(12, 6))
sns.boxplot(x='Price Range', y='Size', data=box_data)
plt.ylabel('')
plt.title('The relationship Between Missing Values & Product Sizes in Track Excavators based on SalePrice');

**Checking the distribution of Missing Values in Wheel Loaders & Track Excavators**

In [ ]:
temp = df.copy()
temp['ProductSize'] = temp['ProductSize'].fillna('Missing')

In [ ]:
temp['ProductSize'] = temp['ProductSize'].replace('Large / Medium', 'Large')

In [ ]:
temp[temp.ProductGroupDesc == 'Track Excavators']['ProductSize'].value_counts(dropna=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax = sns.scatterplot(x=temp[temp.ProductGroupDesc == 'Wheel Loader'].saledate,
                y=temp[temp.ProductGroupDesc == 'Wheel Loader'].SalePrice,
                hue=temp[temp.ProductGroupDesc == 'Wheel Loader'].ProductSize,
                hue_order = ['Large', 'Compact', 'Missing', 'Medium'])
plt.title('SalePrice Distribution of Wheel Loaders based on their Product Size & SaleDate')
plt.xlabel('Sale Date')
plt.ylabel('Sale Price')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1));

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.scatterplot(x=temp[temp.ProductGroupDesc == 'Track Excavators'].saledate,
                y=temp[temp.ProductGroupDesc == 'Track Excavators'].SalePrice,
                hue=temp[temp.ProductGroupDesc == 'Track Excavators'].ProductSize,
                hue_order = ['Missing', 'Mini', 'Small', 'Large'])
plt.title('SalePrice Distribution of Track Excavators based on their Product Size & SaleDate')
plt.xlabel('Sale Date')
plt.ylabel('Sale Price')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1));

According to the two plots above, we see that the missing values in these two groups (`Wheel Loader` and `Track Excavators`) are distributed all over the figure which means they are from various sizes. We could possibly figure out some of these missing values according to the `config` feature we earlier extracted from `fiProductClassDesc`.

In [ ]:
horsepower = {
    'Wheel Loader': {
        40: 'Compact',
        100: 'Small',
        200: 'Medium',
        350: 'Large'
    },
    'Skid Steer Loaders': {
        40: 'Compact',
        100: 'Small',
        200: 'Medium'
    },
    'Track Excavators': {
        40: 'Mini',
        100: 'Small',
        200: 'Medium',
        350: 'Large'
    },
    'Backhoe Loaders': {
        40: 'Compact',
        100: 'Small',
        200: 'Medium'
    },
    'Track Type Tractors': {
        100: 'Small',
        200: 'Medium',
        350: 'Large'
    },
    'Motor Graders': {
        100: 'Medium',
        200: 'Large'
    }
}

operatingcap = {
    'Wheel Loader': {
        2000: 'Compact',
        6000: 'Small',
        15000: 'Medium',
        16000: 'Large'
    },
    'Skid Steer Loaders': {
        1500: 'Compact',
        3000: 'Small',
        5000: 'Medium'
    },
    'Track Excavators': {
        2000: 'Mini',
        5000: 'Small',
        10000: 'Medium',
        20000: 'Large'
    },
    'Backhoe Loaders': {
        2000: 'Compact',
        5000: 'Small',
        10000: 'Medium'
    },
    'Track Type Tractors': {
        10000: 'Small',
        20000: 'Medium',
        40000: 'Large'
    },
    'Motor Graders': {
        10000: 'Medium',
        20000: 'Large'
    }
}

mt = {
    'Wheel Loader': {
        3: 'Compact',
        6: 'Small',
        12: 'Medium',
        13: 'Large'
    },
    'Skid Steer Loaders': {
        0.75: 'Compact',
        1.5: 'Small',
        2.5: 'Medium'
    },
    'Track Excavators': {
        6: 'Mini',
        15: 'Small',
        20: 'Medium',
        45: 'Large'
    },
    'Backhoe Loaders': {
        6: 'Compact',
        10: 'Small',
        14: 'Medium',
        15: 'Large'
    },
    'Track Type Tractors': {
        20: 'Small',
        45: 'Medium',
        46: 'Large'
    },
    'Motor Graders': {
        12: 'Small',
        20: 'Medium',
        21: 'Large'
    }
}

dd = {'Backhoe Loaders':{
        10: 'Compact',
        14: 'Small',
        18: 'Medium',
        24: 'Large'
    }
}

In [ ]:
# Function to determine product size
def size_imputer(row):
    config = row.config
    product = row.ProductGroupDesc

    # If config is not available return nan
    if not isinstance(config, list):
        return np.nan

    else:
        if config[-1] == 'Horsepower':
            size_dict = horsepower[product]
            max_size = max(value for value in config if isinstance(value, (float, int)))
            max_value = min(value if value >= max_size else max(size_dict.keys()) for value in size_dict)
            return size_dict[max_value]
        if config[-1] == 'Metric Tons':
            size_dict = mt[product]
            max_size = max(value for value in config if isinstance(value, (float, int)))
            max_value = min(value if value >= max_size else max(size_dict.keys()) for value in size_dict)
            return size_dict[max_value]
        if config[-1] == 'Operating Capacity':
            size_dict = operatingcap[product]
            max_size = max(value for value in config if isinstance(value, (float, int)))
            max_value = min(value if value >= max_size else max(size_dict.keys()) for value in size_dict)          
            return size_dict[max_value]
        if config[-1] == 'Digging Depth':
            size_dict = dd[product]
            max_size = max(value for value in config if isinstance(value, (float, int)))
            max_value = min(value if value >= max_size else max(size_dict.keys()) for value in size_dict)          
            return size_dict[max_value]

In [ ]:
imputed_size = df.apply(size_imputer, axis=1)

In [ ]:
imputed_size.value_counts(dropna=False)

In [ ]:
# Comparing the imputation with actual available values
imputed_size.compare(df.ProductSize,align_axis=1, keep_shape=False, result_names=('Imputed', 'Actual')).dropna()

There are some differences in categorizations, however, since the values in our imputation dictionaries are according to industry standards, we are not trying to align it with the dataset values since these values could be wrong. Thus, we only impute the missing values in the `ProductSize` column with our imputation result and leave the rest as is.

In [ ]:
# According to our imputation, it's safe to change `Large / Medium` to `Large`
df.ProductSize = df.ProductSize.replace('Large / Medium', 'Large')

In [ ]:
imputed_size = imputed_size.to_dict()
df.ProductSize = df.ProductSize.fillna(imputed_size)

In [ ]:
pd.crosstab(df.ProductSize, df.ProductGroupDesc, margins=True, dropna=False)

In [ ]:
df[df.ProductGroupDesc == 'Wheel Loader'].config.value_counts()

### Inspecting Drive_System

In [ ]:
df.Drive_System.value_counts(dropna=False)

#### Drive System
The drive system of heavy machinery refers to how power is transmitted to the wheels or tracks of the vehicle. For the six types of equipment we have in dataset, the drive systems can vary aS follows:
1. **Wheel Loader:** Typically available in both two-wheel drive (2WD) and four-wheel drive (4WD), but `4WD is more common` due to the need for better traction and stability.
2. **Skid Steer Loaders:** Most commonly equipped with `four-wheel drive (4WD)`, but all-wheel drive (AWD) is also a term sometimes used interchangeably depending on the specific model and manufacturer.
3. **Track Excavators:** These use `tracks` instead of wheels, so the concept of wheel drive systems `doesn't apply`. Instead, they have track drives.
4. **Backhoe Loaders:** Generally available in both two-wheel drive (2WD) and four-wheel drive (4WD), with `4WD being preferred` for off-road capabilities.
5. **Track Type Tractors (Bulldozers):** These also use `tracks` instead of wheels, so they don't fit into the 2WD, 4WD, or AWD categories.
6. **Motor Graders:** Typically available in both two-wheel drive (2WD) and four-wheel drive (4WD), with `4WD versions being more common` for enhanced traction and control.

Also, paying attention to `Motor Graders` and `Backhoe Loaders`, we see that a small number of them are marked as `All Wheel Drive` which according to our research does not exist for these types of machinery, however, since there are only a few records of them, we let them AS IS.

So, we are going to treat `Drive_System` as follows:
1. Imputing missing values as highlighted above.
2. Checking whether our decision for `Backhoe Loaders` is correct.

In [ ]:
pd.crosstab(df.Drive_System, df.ProductGroupDesc,dropna=False, margins=True)

In [ ]:
df.Drive_System = df.Drive_System.replace('No', np.nan)

In [ ]:
missing = df[(df.Drive_System.isna() == True) & (df.ProductGroupDesc == 'Backhoe Loaders')]['SalePrice']
fourw = df[(df.Drive_System == 'Four Wheel Drive') & (df.ProductGroupDesc == 'Backhoe Loaders')]['SalePrice']
twow = df[(df.Drive_System == 'Two Wheel Drive') & (df.ProductGroupDesc == 'Backhoe Loaders')]['SalePrice']

box_data = pd.DataFrame({'Price Range': pd.concat([missing, fourw, twow]),
                         'Drive System': ['Missing']*len(missing)+['4WD']*len(fourw)+['2WD']*len(twow)})
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(x='Price Range', y='Drive System', data=box_data)
plt.title('Comparing Price Range of Missing values in `Drive_System` with the two common groups for `Backhoe Loaders`');

In [ ]:
ds_mapping = {'Wheel Loader': 'Four Wheel Drive',
              'Track Type Tractors': 'Tracks',
              'Track Excavators':'Tracks',
              'Skid Steer Loaders': 'Four Wheel Drive',
              'Motor Graders': 'Four Wheel Drive',
              'Backhoe Loaders': 'Four Wheel Drive'}

def ds_imputer(row):
    if pd.isna(row.Drive_System):
        return ds_mapping[row.ProductGroupDesc]
    else:
        return row.Drive_System
df.Drive_System = df.apply(ds_imputer, axis=1)

### Inspecting Enclosure & Enclosure_Type

In [ ]:
df.Enclosure.value_counts(dropna=False)

#### Enclosure
The enclosure type of heavy equipment refers to the operator's cabin and the level of protection it provides. Here’s a breakdown of the enclosures:

* **EROPS (Enclosed Rollover Protective Structure):** A protective cabin that encloses the operator, providing protection against rollovers and environmental elements.
* **EROPS w AC / EROPS AC:** An EROPS enclosure with air conditioning. These terms can be considered the same as they both indicate an enclosed cabin with AC.
* **OROPS (Open Rollover Protective Structure):** A protective frame around the operator's seat that provides protection against rollovers but does not enclose the operator.
* **NO ROPS:** Indicates the equipment lacks a rollover protective structure.
* **None or Unspecified:** Indicates that no specific information is provided about the enclosure, which might imply no ROPS or that the data is missing.

Enclosures for Each Equipment Type:
* **Wheel Loader:** Can have EROPS (with or without AC) or `OROPS`.
* **Skid Steer Loaders:** Typically have EROPS or `OROPS`, with enclosed versions often having AC.
* **Track Excavators:** Usually have `EROPS` (with or without AC) due to the nature of their work environment.
* **Backhoe Loaders:** Can have EROPS (with or without AC) or `OROPS`.
* **Track Type Tractors (Bulldozers):** Usually have `EROPS` (with or without AC) for protection in harsh conditions.
* **Motor Graders:** Typically have `EROPS` (with or without AC) or OROPS.

Conclusion:
* `OROPS` is a common option for Wheel Loaders, Skid Steer Loaders, Backhoe Loaders, and Motor Graders
* Track Excavators and Track Type Tractors are generally provided with `EROPS` due to safety and environmental protection needs.
* `NO ROPS` is uncommon for modern equipment, might be found in older models or certain types of specialized machinery.

Here's how we are going to Handle the Missing Values/Erroneous types:
1. Combine `EROPS w AC` and `EROPS AC` together
2. Change `None or Unspecified` to `NaN`
3. Check whether we should combine `EROPS w AC` with `EROPS`
4. Change all `NaN` values to the above highlights

In [ ]:
pd.crosstab(df.Enclosure, df.ProductGroupDesc, dropna=False, margins=True)

In [ ]:
missing = df[(df.Enclosure.isna() == True) & (df.ProductGroupDesc == 'Wheel Loader')]['SalePrice']
EROPS = df[((df.Enclosure == 'EROPS') |  (df.Enclosure == 'EROPS w AC')) & (df.ProductGroupDesc == 'Wheel Loader')]['SalePrice']
OROPS = df[(df.Enclosure == 'OROPS') & (df.ProductGroupDesc == 'Wheel Loader')]['SalePrice']

box_data = pd.DataFrame({'Price Range': pd.concat([missing, EROPS, OROPS]),
                         'Enclosure': ['Missing']*len(missing)+['EROPS']*len(EROPS)+['OROPS']*len(OROPS)})
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(x='Price Range', y='Enclosure', data=box_data)
plt.title('Comparing Price Range of Missing values in `Enclosure` with the two common groups for `Wheel Loader`');

In [ ]:
missing = df[(df.Enclosure.isna() == True) & (df.ProductGroupDesc == 'Backhoe Loaders')]['SalePrice']
EROPS = df[((df.Enclosure == 'EROPS') |  (df.Enclosure == 'EROPS w AC')) & (df.ProductGroupDesc == 'Backhoe Loaders')]['SalePrice']
OROPS = df[(df.Enclosure == 'OROPS') & (df.ProductGroupDesc == 'Backhoe Loaders')]['SalePrice']

box_data = pd.DataFrame({'Price Range': pd.concat([missing, EROPS, OROPS]),
                         'Enclosure': ['Missing']*len(missing)+['EROPS']*len(EROPS)+['OROPS']*len(OROPS)})
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(x='Price Range', y='Enclosure', data=box_data)
plt.title('Comparing Price Range of Missing values in `Enclosure` with the two common groups for `Backhoe Loaders`');

In [ ]:
en_mapping = {'EROPS AC': 'EROPS w AC', 'None or Unspecified':np.nan}
df.Enclosure = df.Enclosure.replace(en_mapping)

In [ ]:
EROPS = df[df.Enclosure == 'EROPS']['SalePrice']
EROPSAC = df[df.Enclosure == 'EROPS w AC']['SalePrice']

box_data = pd.DataFrame({'Price Range': pd.concat([EROPS, EROPSAC]),
                         'Enclosure': ['EROPS']*len(EROPS)+['EROPS w AC']*len(EROPSAC)})
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(x='Price Range', y='Enclosure', data=box_data)
plt.title('Comparing Price Range for `EROPS` cabine types with/without AC')
plt.ylabel('');

In [ ]:
en_mapping = {'Wheel Loader': 'OROPS',
              'Track Type Tractors': 'EROPS',
              'Track Excavators':'EROPS',
              'Skid Steer Loaders': 'OROPS',
              'Motor Graders': 'EROPS',
              'Backhoe Loaders': 'OROPS'}
def en_imputer(row):
    if pd.isna(row.Enclosure):
        return en_mapping[row.ProductGroupDesc]
    else:
        return row.Enclosure
df.Enclosure = df.apply(en_imputer, axis=1)

In [ ]:
df.Enclosure_Type.value_counts(dropna=False)

#### Enclosure_Type
The term "Enclosure Type" generally refers to the design and configuration of the operator's cabin or protective structure. 

* **Low Profile:** A design where the operator’s cabin or protective structure is lower to the ground, which can be beneficial for working in areas with height restrictions or where a lower center of gravity is advantageous. More common in `compact equipment` like Skid Steer Loaders and some Wheel Loaders.
* **High Profile:** A design where the operator’s cabin or protective structure is elevated, providing better visibility for the operator and often used in applications where height is not a restriction. Common across most types of heavy equipment, including Wheel Loaders, Track Excavators, Backhoe Loaders, Track Type Tractors, and Motor Graders due to the need for improved operator visibility.

Enclosure Types for Each Equipment Type:
* **Wheel Loader:** Typically can have both low profile and high profile enclosures depending on the model and intended use.
* **Skid Steer Loaders:** Often come in both low profile and high profile variants, with `low profile` being common for `compact` models.
* **Track Excavators:** Generally have high profile enclosures due to the need for better visibility during digging operations.
* **Backhoe Loaders:** Can have both low profile and high profile enclosures.
* **Track Type Tractors (Bulldozers):** Usually have `high profile` enclosures to provide better visibility and control in rugged environments.
* **Motor Graders:** Typically have `high profile` enclosures for better visibility during grading operations.

In [ ]:
pd.crosstab([df.ProductGroupDesc, df.ProductSize], df.Enclosure_Type, dropna=False, margins=True)

In [ ]:
df.ProductSize.value_counts(dropna=False)

### Inspecting Forks

In [ ]:
df.Forks.unique()

In [ ]:
# Checking to see how the nan values are distributed
df[['ProductGroup', 'Forks']].groupby('Forks').value_counts(dropna=False)

In [ ]:
fork_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified'}
df.Forks = df.Forks.replace(fork_mapping)

In [ ]:
df[df.Forks == 'Unspecified']['SalePrice'].describe()

In [ ]:
df[df.Forks == 'Yes']['SalePrice'].describe()

In [ ]:
df.Forks = df.Forks.astype('category')

### Inspecting Pad_Type

In [ ]:
df.Pad_Type.value_counts(dropna=False)

In [ ]:
padtype_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified'}
df.Pad_Type = df.Pad_Type.replace(padtype_mapping)

In [ ]:
df.Pad_Type = df.Pad_Type.astype('category')

### Inspecting Ride_Control

In [ ]:
df.Ride_Control.value_counts(dropna=False)

In [ ]:
ride_control_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified'}
df.Ride_Control = df.Ride_Control.replace(ride_control_mapping)

In [ ]:
df.Ride_Control = df.Ride_Control.astype('category')

### Inspecting Stick

In [ ]:
df.Stick = df.Stick.replace(np.nan, 'Missing')

In [ ]:
# PLotting the SalePrice over the 3 types to see if we can figure out the missing type
missing = df.SalePrice[df.Stick == 'Missing']
standard = df.SalePrice[df.Stick == 'Standard']
extended = df.SalePrice[df.Stick == 'Extended']

box_data = pd.DataFrame({'SalePrice': pd.concat([missing, standard, extended]),
                         'Stick': ['Missing']*len(missing) + ['Standard']*len(standard) + ['Extended']*len(extended)})
sns.violinplot(x='Stick', y='SalePrice', data=box_data)

### Inspecting Transmission

In [ ]:
df.Transmission.value_counts(dropna=False)

In [ ]:
trans_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified', 'AutoShift': 'Autoshift'}
df.Transmission = df.Transmission.replace(trans_mapping)

In [ ]:
df.Transmission = df.Transmission.astype('category')

### Inspectin Turbocharged

In [ ]:
df.Turbocharged.value_counts(dropna=False)

In [ ]:
turbo_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified'}
df.Turbocharged = df.Turbocharged.replace(turbo_mapping)

In [ ]:
df.Turbocharged = df.Turbocharged.astype('category')

### Inspecting Blade_Extension, Blade_Width, Blade_Type

In [ ]:
df.Blade_Extension.value_counts(dropna=False)

In [ ]:
blade_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified'}
df.Blade_Extension = df.Blade_Extension.replace(blade_mapping)

In [ ]:
df.Blade_Extension = df.Blade_Extension.astype('category')

In [ ]:
df.Blade_Width.value_counts(dropna=False)

In [ ]:
df.ProductGroup.unique()

In [ ]:
temp = df[['Blade_Extension', 'Blade_Width', 'Blade_Type']]
temp[temp['Blade_Extension'] == 'Unspecified'].value_counts(dropna=False)

In [ ]:
blade_mapping = {'None or Unspecified': 'Unspecified', np.nan: 'Unspecified'}
df.Blade_Width = df.Blade_Width.replace(blade_mapping)

In [ ]:
df.Blade_Width = df.Blade_Width.astype('category')

### Inspecting Engine_Horsepower

### Inspecting Hydraulics

In [ ]:
df.Hydraulics.value_counts(dropna=False)

### Inspecting Pushblock

### Inspecting Ripper

### Inspecting Scarifier

### Inspecting Tip_control

### Inspecting Tire_Size

### Inspecting Coupler

### Inspecting Coupler_System

### Inspecting Grouser_Tracks

### Inspecting Hydraulics_Flow

### Inspecting Track_Type

### Inspecting Undercarriage_Pad_Width

### Inspecting Stick_Length

In [ ]:
df.Stick_Length.value_counts(dropna=False)

### Inspecting Thumb

### Inspecting Pattern_Changer

In [ ]:
df.Pattern_Changer.value_counts(dropna=False)

### Inspecting Grouser_Type

In [ ]:
df.Grouser_Type.value_counts(dropna=False)

### Inspecting Backhoe_Mounting

### Inspecting Travel_Controls

In [ ]:
df.Travel_Controls.value_counts(dropna=False)

### Inspecting Differential_Type

In [ ]:
df.Differential_Type.value_counts(dropna=False)

### Inspecting Steering_Controls

In [ ]:
df.Steering_Controls.value_counts(dropna=False)

# Feature Selection

In [ ]:
# Forward floating feature selection

# Modelling

# Evaluation

# Hyperparameter Tuning

# Pipeline

# Validation Set

# Test Set

In [ ]:
# Test data
  # importing test data
  # preprocess test data
  # make predictions on test data